### **1. Paramètres**

In [1]:
# Importation des bibliothèques
import os 
import re
import numpy as np
import geopandas as gpd
import fiona
import matplotlib.pyplot as plt
import rasterio
from rasterio.features import rasterize
from rasterio.mask import mask


In [2]:
# Chargement des donnéés
path_vectors = "../data_brut/vector"
shps = fiona.listlayers(path_vectors)

print("Couches disponibles dans le dossier (shps) : ")
for shp in shps:
    print(shp)

Couches disponibles dans le dossier (shps) : 
parcelles_borde_basse_dep82
parcelles_chautagne_dep73
parcelles_dep47
parcelles_gfa_st_pierre_dep82
Peupliers_Dep10_32631


In [3]:
# Charger les couches
dep47_brut = gpd.read_file(path_vectors, layer ='parcelles_dep47')
dep73_brut = gpd.read_file(path_vectors, layer ='parcelles_chautagne_dep73')
dep82_bb_brut = gpd.read_file(path_vectors, layer ='parcelles_borde_basse_dep82')
dep82_sp_brut = gpd.read_file(path_vectors, layer ='parcelles_gfa_st_pierre_dep82')
dep_10_brut = gpd.read_file(path_vectors, layer = "Peupliers_Dep10_32631")

### **2. Nettoyer le dep47**

In [4]:
# Faire une copie de la couche
dep47 = dep47_brut.copy()
print(dep47.columns)

Index(['id', 'commune', 'prefixe', 'section', 'numero', 'contenance',
       'created', 'updated', 'Structure_', 'Nom', 'Pr�nom', 'Commune_pl',
       'Code_Insee', 'Pr�fixe', 'Section_1', 'N�', 'Subdivisio', 'Surface_pa',
       'Surface_pl', 'Ann�e_pl', 'Cultivars', 'Densit�', 'ID_peuplie',
       'comment', 'geometry'],
      dtype='object')


#### **2.1 Sélectionner les colonnes**

In [5]:
# Sélectionnér les colonnes à conserver
col_dep47 = ['id',
             'Ann�e_pl',
             'Cultivars',
             'Densit�',
             'ID_peuplie',
             'comment',
             'geometry'
             ]

# Filtrer les colonnes pour garder seulement celles qui sont nécessaires
dep47 = dep47[col_dep47]

# Afficher les colonnes filtrées
print(dep47.columns)

Index(['id', 'Ann�e_pl', 'Cultivars', 'Densit�', 'ID_peuplie', 'comment',
       'geometry'],
      dtype='object')


#### **2.2 Renommer les colonnes**

In [6]:
# Créer un dictionnaire avec les nouveaux noms de colonnes
nom_col_dep47 = {'id' : 'id_47',
                 'Ann�e_pl' : 'annee_plan',
                 'Cultivars' : 'cultivar',
                 'Densit�' : 'densite',
                 'ID_peuplie' : 'id_peulplie'
                 }

# Renommer les colonnes
dep47.rename(columns=nom_col_dep47, inplace=True)
# Afficher les nouvelles colonnes pour vérification
print("Colonnes après nettoyage et renommage:")
print(dep47.columns)

Colonnes après nettoyage et renommage:
Index(['id_47', 'annee_plan', 'cultivar', 'densite', 'id_peulplie', 'comment',
       'geometry'],
      dtype='object')


#### **2.3 Sélection de parcelles avec des cultivars uniques**

In [7]:
# Boucle pour afficher les valeurs uniques de la colonne 'cultivars'
uniques_dep47 = dep47['cultivar'].unique()
for unique in uniques_dep47:
    print(unique)

Raspalje - I45/51
I45/51
Koster - I45/51
Alcinde
Koster
Dorskamp
I45/51-Dorskamd-Alcinde
Raspalje
Polargo
Raspalje-Polargo-I45/51
Koster-Raspalje
Koster-I45/51
I214
Polargo - Koster
Koster-Polargo-Rona
Polargo - Lambro
Polargo-Raspalje-I45/51
Koster-Polargo-I45/51
Raspalje-Alcinde
Soligo
Raspalje - Soligo
I45/51 - Raspalje
I45/51 - rASPALJE
I214 - LENA
Polarco-I45/51-Dvina
Polargo - A4A
Koster - Polargo
Raspalje-Koster
Dorskamp-I45/5
Dorskamp-Korster
Koster - Raspalje


In [8]:
# Fonction pour vérifier si la valeur contient exactement un cultivar
def seul_cultivar(valeur):
    # Vérifie d'abord si la valeur est une chaîne de caractères
    if not isinstance(valeur, str):
        return False
    # Si la cellule est vide, contient 'nan', ou contient seulement des espaces, retourner False
    if not valeur or valeur.strip().lower() == "nan" or valeur.strip() == "":
        return False
    # Si la valeur contient une virgule, les mots 'melange', 'essai', 'divers', 'et', ou 'Vesten Dellinois', retourner False
    if ',' in valeur or re.search(r'\b(melange|essai|essais|divers|et)\b', valeur, re.IGNORECASE) or "vesten dellinois".lower() in valeur.lower():
        return False
    # Exclure 'Noyer/robinier', mas NÃO exclua 'I45/51'
    if re.search(r'noyer/robinier', valeur.lower()):
        return False
    # Si la valeur contient des numéros au format d'essai (ex. 1000-1, 0685-16), retourner False
    if re.search(r'\d{3,4}-\d{1,2}', valeur):
        return False
    # Supprime les espaces superflus et convertit en minuscules
    valeur = valeur.strip().lower()
    # Divise la chaîne sur des tirets (avec ou sans espace) ou des espaces
    cultivars = re.split(r'\s*-\s*', valeur)
    # Vérifie s'il y a exactement un cultivar unique
    return len(set(cultivars)) == 1


# Filtrer les lignes avec un seul cultivar dans la colonne 'cultivar'
dep47_cultivar_unique = dep47[dep47['cultivar'].apply(seul_cultivar)]

# Affiche les premières lignes pour vérifier le résultat
print("Nombre total de lignes avant filtrage :", len(dep47))
print("Nombre de lignes avec un seul cultivar :", len(dep47_cultivar_unique))

# Boucle pour afficher les valeurs uniques de la colonne 'cultivars'
uniques_dep47_single_cultivar = dep47_cultivar_unique['cultivar'].unique()
for unique_clean in uniques_dep47_single_cultivar:
    print(unique_clean)
    
# Afficher les premières lines
dep47_cultivar_unique.head()

Nombre total de lignes avant filtrage : 114
Nombre de lignes avec un seul cultivar : 72
I45/51
Alcinde
Koster
Dorskamp
Raspalje
Polargo
I214
Soligo


,id_47,annee_plan,cultivar,densite,id_peulplie,comment,geometry
1,472770000A0738,2010,I45/51,200,472770000A0738,redessinee,"POLYGON ((460197.741 6381459.148, 460186.836 6..."
3,472770000A0737,2010,I45/51,200,472770000A0737,redessinee,"POLYGON ((460201.636 6381455.838, 460208.858 6..."
4,472770000A0748,2010,Alcinde,200,472770000A0748,ok,"POLYGON ((460182.340 6381608.834, 460173.209 6..."
5,470100000A0042,2011,Koster,200,470100000A0042,redessinee seule une partie conservee,"POLYGON ((461223.107 6367436.887, 461216.981 6..."
6,472770000A0739,2010,I45/51,200,472770000A0739,ok,"POLYGON ((460133.672 6381680.566, 460126.580 6..."


### **3. Nettoyer le dep73**

In [9]:
# Faire une copie de la couche
dep73 = dep73_brut.copy()
print(dep73.columns)
dep73.head(2)

Index(['fid', 'iidtn_frt', 'llib_frt', 'ccod_prf', 'cinse_dep', 'Annee_plan',
       'Cultivar1', 'Age1', 'Annee_pl_1', 'Cultivar2', 'Age2', 'geometry'],
      dtype='object')


,fid,iidtn_frt,llib_frt,ccod_prf,cinse_dep,Annee_plan,Cultivar1,Age1,Annee_pl_1,Cultivar2,Age2,geometry
0,1.0,F20305Y,For�t domaniale de Chautagne,112,73,2009,"Raspage, Gaver, Dorskamp, A4A, Soligo",15.0,NaN,NaN,NaN,"POLYGON ((920031.391 6533890.964, 920009.900 6..."
1,2.0,F20305Y,For�t domaniale de Chautagne,113,73,2019,"Triplo, Vesten",5.0,NaN,NaN,NaN,"POLYGON ((920057.400 6534027.100, 920241.814 6..."


#### **3.1 Sélectionner les colonnes**

In [10]:
# Sélectionnér les colonnes à conserver
col_dep73 = ['fid',
             'Annee_plan',
             'Cultivar1',
             'geometry'
             ]

# Filtrer les colonnes pour garder seulement celles qui sont nécessaires
dep73 = dep73[col_dep73]

# Afficher les colonnes filtrées
print(dep73.columns)

Index(['fid', 'Annee_plan', 'Cultivar1', 'geometry'], dtype='object')


#### **3.2 Renommer les colonnes**

In [11]:
# Créer un dictionnaire avec les nouveaux noms de colonnes
nom_col_dep73 = {'fid' : 'id_73',
                 'Annee_plan' : 'annee_plan',
                 'Cultivar1' : 'cultivar'
                 }

# Renommer les colonnes
dep73.rename(columns=nom_col_dep73, inplace=True)

# Afficher les nouvelles colonnes pour vérification
print("Colonnes après nettoyage et renommage:")
print(dep73.columns)

Colonnes après nettoyage et renommage:
Index(['id_73', 'annee_plan', 'cultivar', 'geometry'], dtype='object')


#### **3.3 Sélection de parcelles avec des cultivars uniques**

In [12]:
# Boucle pour afficher les valeurs uniques de la colonne 'cultivars'
uniques_dep73 = dep73['cultivar'].unique()
for unique in uniques_dep73:
    print(unique)

Raspage, Gaver, Dorskamp, A4A, Soligo
Triplo, Vesten
Flevo
Dorskamp
Koster
Soligo, Dellinois
Soligo
Flevo, Dorskamp
Dorskamp, Raspage, Flevo
Gaver, Flevo, Raspage
Flevo, Dorskamp, A4A
Vesten
Cemagref
Dorskamp, Gaver
Triplo
Koster, Soligo, A4A
Gaver, Triplo, Soligo
Gaver, Dorskamp, Soligo
Vesten Dellinois
Flevo, Gaver
Dorskamps, Gaver
Dorskamp, Bara, A4A
A4A, Koster, Soligo, Dorskamp, Flevo
Koster, Dorskamp, Flévo
Gaver, Triplo
Dorskamp, Flevo
Koster, Dorskamp, Flevo
Triplo, Koster
Triplo, Bara, A4A, Soligo
Soligo, Delinoi, Diva, Dano
Rona, Soligo, Flevo Dorskamp
Raspage
Soligo, Diva
Soligo, Triplo, Tucano
Bara, Flevo, Raspage, Koster
Dorskamp, Flevo, Raspage
Raspage, Flevo
Flevo, Dorskamp, Gaver
Flevo, Gaver, Dorskamp
Gaver, Raspage, Triplo, HooGwor
Soligo, Triplo, Koster
Dorskamp, FCBA
Koster, A4A
Faux Gaver, Dorskamp
Dellinois, Koster
Dorskamp, Triplo, Soligo
Faux Gaver
Hoo Gwor, Gaver, A4A
9 Clones
Triplo, Flevo, Gaver
Dellinois, Tucano
Rona, Tucano


In [13]:
# Filtrer les lignes avec un seul cultivar dans la colonne 'cultivars'
dep73_cultivar_unique = dep73[dep73['cultivar'].apply(seul_cultivar)]

# Affiche les premières lignes pour vérifier le résultat
print("Nombre total de lignes avant filtrage :", len(dep73))
print("Nombre de lignes avec un seul cultivar :", len(dep73_cultivar_unique))

# Boucle pour afficher les valeurs uniques de la colonne 'cultivars'
uniques_dep73_single_cultivar = dep73_cultivar_unique['cultivar'].unique()
for unique_73clean in uniques_dep73_single_cultivar:
    print(unique_73clean)
    
# Afficher les premières lines
dep73_cultivar_unique.head()

Nombre total de lignes avant filtrage : 73
Nombre de lignes avec un seul cultivar : 26
Flevo
Dorskamp
Koster
Soligo
Vesten
Cemagref
Triplo
Raspage
Faux Gaver
9 Clones


,id_73,annee_plan,cultivar,geometry
2,3.0,2000,Flevo,"POLYGON ((919474.367 6532213.335, 919673.281 6..."
3,4.0,2002,Dorskamp,"POLYGON ((919495.629 6532313.795, 919689.900 6..."
4,5.0,2022,Koster,"POLYGON ((918327.900 6530920.200, 918327.900 6..."
6,7.0,2019,Soligo,"POLYGON ((919465.518 6533574.488, 919267.636 6..."
12,13.0,2022,Vesten,"POLYGON ((918138.800 6530729.700, 918129.951 6..."


### **4. Nettoyer le dep82 - Borde Basse**

In [14]:
# Faire une copie de la couche de la région Borde Basse
dep82_bb = dep82_bb_brut.copy()
print(dep82_bb.columns)
dep82_bb.head(2)

Index(['Parcelle', 'Sous_parce', 'Type_peupl', 'Essence', 'Année_pla',
       'Surface', 'Surf_ha', 'ID_user', 'geometry'],
      dtype='object')


,Parcelle,Sous_parce,Type_peupl,Essence,Année_pla,Surface,Surf_ha,ID_user,geometry
0,NaN,NaN,NaN,Raspalje,2020,NaN,0.262,1,"POLYGON ((544260.145 6333595.096, 544320.860 6..."
1,NaN,NaN,NaN,Noyer hybride,1990,NaN,5.528,2,"POLYGON ((544224.090 6333655.140, 544235.930 6..."


#### **41. Sélectionner les colonnes dep82 - Borde Basse**

In [15]:
# Sélectionnér les colonnes à conserver
col_dep82_bb = ['ID_user',
                'Essence',
                'Année_pla',
                'geometry'
                ]

# Filtrer les colonnes pour garder seulement celles qui sont nécessaires
dep82_bb = dep82_bb[col_dep82_bb]

# Afficher les colonnes filtrées
print(dep82_bb.columns)
dep82_bb.head(2)

Index(['ID_user', 'Essence', 'Année_pla', 'geometry'], dtype='object')


,ID_user,Essence,Année_pla,geometry
0,1,Raspalje,2020,"POLYGON ((544260.145 6333595.096, 544320.860 6..."
1,2,Noyer hybride,1990,"POLYGON ((544224.090 6333655.140, 544235.930 6..."


#### **4.2 Renommer les colonnes**

In [16]:
# Créer un dictionnaire avec les nouveaux noms de colonnes
nom_col_dep82_bb = {'ID_user' : 'id_82_bb',
                 'Année_pla' : 'annee_plan',
                 'Essence' : 'cultivar'
                 }

# Renommer les colonnes
dep82_bb.rename(columns=nom_col_dep82_bb, inplace=True)

# Afficher les nouvelles colonnes pour vérification
print("Colonnes après nettoyage et renommage:")
print(dep82_bb.columns)

Colonnes après nettoyage et renommage:
Index(['id_82_bb', 'cultivar', 'annee_plan', 'geometry'], dtype='object')


#### **4.3 Sélection des parcelles avec une seul cultivar**

In [17]:
# Boucle pour afficher les valeurs uniques de la colonne 'cultivars'
uniques_dep82_bb = dep82_bb['cultivar'].unique()
for unique_82_bb in uniques_dep82_bb:
    print(unique_82_bb)

Raspalje
Noyer hybride
Futaie melangee
Cèdre
I 45/51
Melange PEU/NOY
Melange NOY/ROB
Robinier
I 214
Polargo
Koster
Rona
AF8
Triplo
Soligo
I 2014
A4A
Alcinde
Lambro
Degrosso
Dano
Garo
Ludo
San martino
Tucano
Diva
Senna
Imola
Moleto
Moncalvo
Aleramo


In [18]:
# Filtrer les lignes avec un seul cultivar dans la colonne 'cultivar'
dep82_bb_cultivar_unique = dep82_bb[dep82_bb['cultivar'].apply(seul_cultivar)]

# Affiche les premières lignes pour vérifier le résultat
print("Nombre total de lignes avant filtrage :", len(dep82_bb))
print("Nombre de lignes avec un seul cultivar :", len(dep82_bb_cultivar_unique))

# Boucle pour afficher les valeurs uniques de la colonne 'cultivars'
uniques_dep82_bb_single_cultivar = dep47_cultivar_unique['cultivar'].unique()
for unique_dep82_bb_single_cultivar in uniques_dep82_bb_single_cultivar:
    print(unique_dep82_bb_single_cultivar)
    
# Afficher les premières lines
dep82_bb_cultivar_unique.head()

Nombre total de lignes avant filtrage : 54
Nombre de lignes avec un seul cultivar : 52
I45/51
Alcinde
Koster
Dorskamp
Raspalje
Polargo
I214
Soligo


,id_82_bb,cultivar,annee_plan,geometry
0,1,Raspalje,2020,"POLYGON ((544260.145 6333595.096, 544320.860 6..."
1,2,Noyer hybride,1990,"POLYGON ((544224.090 6333655.140, 544235.930 6..."
2,3,Futaie melangee,2021,"POLYGON ((544686.859 6333334.250, 544702.614 6..."
3,4,Cèdre,2009,"POLYGON ((544827.303 6333279.698, 544889.980 6..."
4,5,I 45/51,2012,"POLYGON ((544817.900 6333572.410, 544847.780 6..."


### **5. Nettoyer le dep82 - Saint Pierre**

In [19]:
# Faire une copie de la couche de la région Borde Basse
dep82_sp = dep82_sp_brut.copy()
print(dep82_sp.columns)
dep82_sp.head(2)

Index(['parcelle_F', 'Type_ppt', 'Sous_parce', 'Espece', 'Annee', 'Surface_ha',
       'ID_user', 'geometry'],
      dtype='object')


,parcelle_F,Type_ppt,Sous_parce,Espece,Annee,Surface_ha,ID_user,geometry
0,NaN,NaN,NaN,Raspalje,2017,0.819,1,"POLYGON ((559437.988 6303972.233, 559516.300 6..."
1,NaN,NaN,NaN,Soligo,2017,1.017,2,"POLYGON ((559453.650 6303798.828, 559421.740 6..."


#### **5.1 Sélectionner les colonnes**

In [20]:
# Sélectionnér les colonnes à conserver
col_dep82_sp = ['ID_user',
                'Espece',
                'Annee',
                'geometry'
                ]

# Filtrer les colonnes pour garder seulement celles qui sont nécessaires
dep82_sp = dep82_sp[col_dep82_sp]

# Afficher les colonnes filtrées
print(dep82_sp.columns)
dep82_sp.head(3)

Index(['ID_user', 'Espece', 'Annee', 'geometry'], dtype='object')


,ID_user,Espece,Annee,geometry
0,1,Raspalje,2017,"POLYGON ((559437.988 6303972.233, 559516.300 6..."
1,2,Soligo,2017,"POLYGON ((559453.650 6303798.828, 559421.740 6..."
2,3,Koster,2017,"POLYGON ((559393.238 6303793.234, 559383.169 6..."


#### **5.2 Renommer les colonnes**

In [21]:
# Créer un dictionnaire avec les nouveaux noms de colonnes
nom_col_dep82_sp = {'ID_user' : 'id_82_sp',
                  'Annee' : 'annee_plan',
                 'Espece' : 'cultivar'
                 }

# Renommer les colonnes
dep82_sp.rename(columns=nom_col_dep82_sp, inplace=True)

# Afficher les nouvelles colonnes pour vérification
print("Colonnes après nettoyage et renommage:")
print(dep82_sp.columns)

Colonnes après nettoyage et renommage:
Index(['id_82_sp', 'cultivar', 'annee_plan', 'geometry'], dtype='object')


#### **5.3 Sélection des parcelles**

In [22]:
# Boucle pour afficher les valeurs uniques de la colonne 'cultivars'
uniques_dep82_sp = dep82_sp['cultivar'].unique()
for unique_82_sp in uniques_dep82_sp:
    print(unique_82_sp)

Raspalje
Soligo
Koster
Moncalvo
Diva
Tucano
AF8
BIC
Eiffel
Missouri
0685-16
27208-16
AF13
Hoogvorst
Dvina
Oglio
Dender
Marke
Divers GIS
Aleramo
San martino
Divers
Senna
Rona et Raspalje
San Martino
Oglio et Raspalje
Rona
Moleto
Essais Alasia BC
Noyer hybride
Noyer commun
I 214
melange
Dorskamp
Vesten
Alcinde
Polargo
Oudenberg
Pannonia
Kopecky
Lambro
Lena
Robinier
Triplo
Imola
Taro
Dano
Delvignac
Noyer/robinier
Essai
Hoogorst
Ameramo
Sprint
Agora
Palorn
Nikos
Turbo
700-1
1000-1
1000-2
1000-3
1000-4
nan
Divers 82 19 001
Divers 82 18 001


In [23]:
# Filtrer les lignes avec un seul cultivar dans la colonne 'cultivar'
dep82_sp_cultivar_unique = dep82_sp[dep82_sp['cultivar'].apply(seul_cultivar)]

# Affiche les premières lignes pour vérifier le résultat
print("Nombre total de lignes avant filtrage :", len(dep82_sp))
print("Nombre de lignes avec un seul cultivar :", len(dep82_sp_cultivar_unique))

# Boucle pour afficher les valeurs uniques de la colonne 'cultivars'
uniques_dep82_sp_single_cultivar = dep82_sp_cultivar_unique['cultivar'].unique()
for unique_dep82_sp_single_cultivar in uniques_dep82_sp_single_cultivar:
    print(unique_dep82_sp_single_cultivar)
    
# Afficher les premières lines
dep82_sp_cultivar_unique.head()

Nombre total de lignes avant filtrage : 259
Nombre de lignes avec un seul cultivar : 237
Raspalje
Soligo
Koster
Moncalvo
Diva
Tucano
AF8
BIC
Eiffel
Missouri
AF13
Hoogvorst
Dvina
Oglio
Dender
Marke
Aleramo
San martino
Senna
San Martino
Rona
Moleto
Noyer hybride
Noyer commun
I 214
Dorskamp
Vesten
Alcinde
Polargo
Oudenberg
Pannonia
Kopecky
Lambro
Lena
Robinier
Triplo
Imola
Taro
Dano
Delvignac
Hoogorst
Ameramo
Sprint
Agora
Palorn
Nikos
Turbo


,id_82_sp,cultivar,annee_plan,geometry
0,1,Raspalje,2017,"POLYGON ((559437.988 6303972.233, 559516.300 6..."
1,2,Soligo,2017,"POLYGON ((559453.650 6303798.828, 559421.740 6..."
2,3,Koster,2017,"POLYGON ((559393.238 6303793.234, 559383.169 6..."
3,4,Moncalvo,2022,"POLYGON ((559765.018 6304164.920, 559752.890 6..."
4,5,Diva,2022,"POLYGON ((559729.105 6304304.045, 559715.707 6..."


### **6. Nettoyer le dep10**

In [24]:
# Faire une copie de la couche de département 10
dep10 = dep_10_brut.copy()
print(dep10.columns)
dep10.head(2)

Index(['ID', 'Annee', 'd_Peuplem', 'd_essenc_1', 'd_essenc_2', 'geometry'], dtype='object')


,ID,Annee,d_Peuplem,d_essenc_1,d_essenc_2,geometry
0,1,2015,Peupleraie,Soligo,NaN,"POLYGON ((536321.974 5373586.757, 536461.778 5..."
1,2,2011,Peupleraie,Polargo,NaN,"POLYGON ((536441.413 5373956.204, 536404.209 5..."


#### **6.1. Sélectionner les colonnes de dep10**

In [25]:
# Sélectionnér les colonnes à conserver
col_dep10 = ['ID',
                'd_essenc_1',
                'Annee',
                'geometry'
                ]

# Filtrer les colonnes pour garder seulement celles qui sont nécessaires
dep10 = dep10[col_dep10]

# Afficher les colonnes filtrées
print(dep10.columns)
dep10.head(3)

Index(['ID', 'd_essenc_1', 'Annee', 'geometry'], dtype='object')


,ID,d_essenc_1,Annee,geometry
0,1,Soligo,2015,"POLYGON ((536321.974 5373586.757, 536461.778 5..."
1,2,Polargo,2011,"POLYGON ((536441.413 5373956.204, 536404.209 5..."
2,3,I214,1996,"POLYGON ((536724.852 5374336.503, 536739.049 5..."


#### **6.2 Renommer les colonnes**

In [26]:
# Créer un dictionnaire avec les nouveaux noms de colonnes
nom_col_dep10 = {'ID' : 'id_10',
                  'Annee' : 'annee_plan',
                 'd_essenc_1' : 'cultivar'
                 }

# Renommer les colonnes
dep10.rename(columns=nom_col_dep10, inplace=True)

# Afficher les nouvelles colonnes pour vérification
print("Colonnes après nettoyage et renommage:")
print(dep10.columns)

Colonnes après nettoyage et renommage:
Index(['id_10', 'cultivar', 'annee_plan', 'geometry'], dtype='object')


#### **6.3 Sélection des parcelles**

In [27]:
# Boucle pour afficher les valeurs uniques de la colonne 'cultivars'
uniques_dep10 = dep10['cultivar'].unique()
for unique_dep10 in uniques_dep10:
    print(unique_dep10)

Soligo
Polargo
I214
Koster
Boelare
nan
Rona
Fritzy pauley
Moncalvo
Triplo
Flevo
Trichobel
Albelo
A4a
Degrosso
Vesten
Brenta
Diva


In [28]:
# Filtrer les lignes avec un seul cultivar dans la colonne 'cultivar'
dep10_cultivar_unique = dep10[dep10['cultivar'].apply(seul_cultivar)]

# Affiche les premières lignes pour vérifier le résultat
print("Nombre total de lignes avant filtrage :", len(dep10))
print("Nombre de lignes avec un seul cultivar :", len(dep10_cultivar_unique))

# Boucle pour afficher les valeurs uniques de la colonne 'cultivars'
uniques_dep10_single_cultivar = dep10_cultivar_unique['cultivar'].unique()
for unique_dep10_single_cultivar in uniques_dep10_single_cultivar:
    print(unique_dep10_single_cultivar)
    
# Afficher les premières lines
dep10_cultivar_unique.head()

Nombre total de lignes avant filtrage : 90
Nombre de lignes avec un seul cultivar : 82
Soligo
Polargo
I214
Koster
Boelare
Rona
Fritzy pauley
Moncalvo
Triplo
Flevo
Trichobel
Albelo
A4a
Degrosso
Vesten
Brenta
Diva


,id_10,cultivar,annee_plan,geometry
0,1,Soligo,2015,"POLYGON ((536321.974 5373586.757, 536461.778 5..."
1,2,Polargo,2011,"POLYGON ((536441.413 5373956.204, 536404.209 5..."
2,3,I214,1996,"POLYGON ((536724.852 5374336.503, 536739.049 5..."
3,4,I214,1997,"POLYGON ((536019.656 5373695.181, 536023.535 5..."
4,5,I214,2002,"POLYGON ((536202.168 5373793.123, 536274.102 5..."
